# ![RAG](slides/langchain.png)


# ![RAG](slides/image_rag.png)


# LangChain. Naive RAG


# ![RAG](slides/image_langchain.png)



## Settings

Вам нужно будет установить несколько пакетов и установить ваш OpenAI API ключ как переменную окружения с именем `OPENAI_API_KEY`:

In [1]:
%pip install -qU langchain langchain-openai langchain-chroma beautifulsoup4
%pip install -qU langchain-community pypdf


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv('.env', override=True)

True

In [ ]:
import os

SAFE_KEYS = ('LANGCHAIN_TRACING_V2','LANGCHAIN_ENDPOINT','LANGCHAIN_PROJECT','LANGSMITH_ENDPOINT','LANGSMITH_PROJECT')
for k in SAFE_KEYS:
    if k in os.environ:
        print(f'{k}: {os.environ[k]}')
if os.getenv('LANGCHAIN_API_KEY'):
    print('LANGCHAIN_API_KEY: <set>')
if os.getenv('LANGSMITH_API_KEY'):
    print('LANGSMITH_API_KEY: <set>')


## Index


# ![RAG](slides/image_vectorstore.png)


### Загрузка документов

### Loader
Воспользуемся PyPDFLoader для загрузки документа:

In [4]:

from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

loader = PyPDFLoader(
    file_path = "./documents/smirnoff_ai.pdf",
    mode = "page",
    extraction_mode = "plain"
     # headers = None
    # password = None,
    # pages_delimiter = "",
    # extract_images = True,
    # images_parser = RapidOCRBlobParser()
)


Теперь прочитаем каждую страницу документа и сохраним в список:

In [5]:

pages = []
for page in loader.load():
    pages.append(page) 
       
len(pages)

14

Давайте посмотрим на первую страницу документа:

In [6]:
print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

{'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'SMIRNOFF_AI. Профиль с портфолио', 'source': './documents/smirnoff_ai.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}

Профессиональная  разработка  ИИ-решений  для  бизнеса  
Профессиональная  разработка  и  внедрение  систем  искусственного  интеллекта,  ИИ-ассистентов,  ИИ-агентов,  ИИ-ботов  и  других  решений  на  базе  генеративного  искусственного  интеллекта  для  частных  клиентов,  малого  и  среднего  бизнеса.  
УСЛУГИ —  Разработка  AI- решений/агентов/ботов  для  автоматизации  бизнеса  —  Аудит  процессов  и  разработка  дорожной  карты  внедрения  AI  —  Консалтинг  по  разработке  и  внедрению  AI  —  Обучение  разработке  решений  на  базе  генеративного  ИИ  
КЛЮЧЕВЫЕ
 
КОМПЕТЕНЦИИ
 
И
 
ОПЫТ
 
ИИ-ассистенты  Разработка  AI- ассистентов,  повышающих  эффективность  выполнения  рабочих  задач  сотрудников 
—  ИИ-ассистент  сотрудника  медиа-агентства  MediaWise  

### Splitter

Далее мы разбиваем его на меньшие chunks. Воспользуемся RecursiveCharacterTextSplitter:

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(pages)
len(all_splits)

46

### Vectorstore and embeddings

Теперь загрузим наши чанки в **векторное хранилище**. Одновременно с этим для каждого чанка мы создадим соответствующий векторный образ (эмбеддинг).

Будем использовать эмбеддинг модель из OpenAI:

In [8]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Будем использовать векторное хранилище InMemoryVectorStore (в памяти)

In [9]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

vector_store = InMemoryVectorStore.from_documents(
    all_splits,
    embedding=OpenAIEmbeddings(model="openai/text-embedding-3-large"), # text-embedding-3-large
)


## Retrieval

Теперь создадим retriever из нашего vectorstore для поиска по нему:

In [10]:
# k is the number of chunks to retrieve
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

question = "Вы проводите консультации?"
chunks = retriever.invoke(question)

len(chunks)

3

In [11]:
print(f"{chunks[0].metadata}\n")
print(chunks[0].page_content)

{'producer': 'Skia/PDF m136 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'SMIRNOFF_AI. Профиль с портфолио', 'source': './documents/smirnoff_ai.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}

УСЛУГИ —  Разработка  AI- решений/агентов/ботов  для  автоматизации  бизнеса  —  Аудит  процессов  и  разработка  дорожной  карты  внедрения  AI  —  Консалтинг  по  разработке  и  внедрению  AI  —  Обучение  разработке  решений  на  базе  генеративного  ИИ  
КЛЮЧЕВЫЕ
 
КОМПЕТЕНЦИИ
 
И
 
ОПЫТ
 
ИИ-ассистенты  Разработка  AI- ассистентов,  повышающих  эффективность  выполнения  рабочих  задач  сотрудников


Мы видим, что вызов retriever выше возвращает некоторые части документа, которую наш чат-бот может использовать в качестве контекста при ответе на вопросы. И теперь у нас есть retriever, который может возвращать связанные данные из документа!

## Augmented generation

# ![RAG](slides/image_prompt_augmentation.png)


Напишем функцию для объединения чанков в одну строку, чтобы позже использовать ее в prompt для задания контекста

In [12]:
# Concat chunks into a single string to insert into the prompt
def format_chunks(chunks):
    return "\n\n".join(chunk.page_content for chunk in chunks)

chunks_context = format_chunks(chunks)

### PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

SYSTEM_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the user question.
If you don't find the answer in provided context strictly say 'Я не нашел ответа на ваш вопрос!'.
Use three sentences maximum and keep the answer concise.

Context:
{context}
"""

question_answering_prompt = ChatPromptTemplate([
        ("system", SYSTEM_TEMPLATE),
        ("human", "{question}"),
    ]
)

### Chat (LLM)

Будем использовать модель от OpenAI

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="openai/gpt-oss-20b:free", temperature=0.9)

Попросим LLM ответить на вопрос по найденным чанкам в контексте

In [15]:
from langchain_core.messages import HumanMessage

llm.invoke(question_answering_prompt.invoke(
    {
        "context": chunks_context,
        "question": question,
    })
)

AIMessage(content='Да, в рамках наших услуг мы проводим консалтинг по разработке и внедрению AI‑решений.  \nМы помогаем аудитировать процессы, разрабатывать дорожные карты и обучать команду.  \nЕсли нужна персональная консультация, уточните детали, и мы подберём подходящий пакет.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 516, 'total_tokens': 644, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-20b:free', 'system_fingerprint': None, 'id': 'gen-1762461315-zDJ7EtlulmhwifdPPr7d', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--f3556f1a-4d12-473a-bc39-b6c3885b9ac9-0', usage_metadata={'input_tokens': 516, 'output_tokens': 128, 'total_tokens': 644, 'input_token_details': {}, 'output_token_details': {}})

Выглядит хорошо! Для сравнения попробуем без контекстных документов и сравним результат:

In [16]:
llm.invoke(question_answering_prompt.invoke(
    {
        "context": "",
        "question": question,
    })
)

AIMessage(content='Я не нашел ответа на ваш вопрос!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 135, 'total_tokens': 247, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'openai/gpt-oss-20b:free', 'system_fingerprint': None, 'id': 'gen-1762461328-3MpNeA7VTIksEbrHohWv', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--8d1b3862-9500-45e8-bd4b-c87a4dfdaff6-0', usage_metadata={'input_tokens': 135, 'output_tokens': 112, 'total_tokens': 247, 'input_token_details': {}, 'output_token_details': {}})

Браво! Мы с вами создали первую RAG систему!


## Retrieval chains


Давайте теперь создадим цепочку chain, которая будет принимать вопрос от пользователя и возвращать ответ на основе найденных чанков.

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_chunks, "question": RunnablePassthrough()}
    | question_answering_prompt
    | llm
    | StrOutputParser()
)

Посмотрите, как удобно теперь использовать - одна строчка кода и мы получаем ответ на наш вопрос:

In [18]:
# Question
answer = rag_chain.invoke(question)

answer

'Да, мы проводим консультации по разработке и внедрению ИИ‑решений.  \nНаша команда проводит аудит процессов, разрабатывает дорожную карту внедрения и предоставляет рекомендации по оптимизации.  \nМы помогаем ускорить переход на ИИ и повысить эффективность вашего бизнеса.'

Выглядит хорошо!


## Conversation mode


### Conversation chat

Создадим шаблон промпта для учета всей истории диалога

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage

CONVERSATION_SYSTEM_TEMPLATE = """
You are an assistant for question-answering tasks. Answer the user's questions based on the conversation history and below context retrieved for the last question. Answer 'Я не нашел ответа на ваш вопрос!' if you don't find any information in the context. Use three sentences maximum and keep the answer concise.\n\nContext retrieved for the last question:\n\n{context}
"""


conversational_answering_prompt = ChatPromptTemplate(
    [
        ("system", CONVERSATION_SYSTEM_TEMPLATE),
        ("placeholder", "{messages}")
    ]
)

conversational_answering_prompt.invoke(
    {
        "context": "Чанки контекста",
        "messages": [
            HumanMessage(content=question)
        ]
    }
)

ChatPromptValue(messages=[SystemMessage(content="\nYou are an assistant for question-answering tasks. Answer the user's questions based on the conversation history and below context retrieved for the last question. Answer 'Я не нашел ответа на ваш вопрос!' if you don't find any information in the context. Use three sentences maximum and keep the answer concise.\n\nContext retrieved for the last question:\n\nЧанки контекста\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='Вы проводите консультации?', additional_kwargs={}, response_metadata={})])

Реализуем цепочку ответа на последний заданный вопрос.

In [20]:
from typing import Dict
from langchain_core.runnables import RunnablePassthrough

def get_last_message_for_retriever_input(params: Dict):
    return params["messages"][-1].content


In [21]:
last_message_retriever_chain = get_last_message_for_retriever_input | retriever | format_chunks 
last_message_retriever_chain.invoke({"messages": [
            HumanMessage(content=question)
        ]})

'УСЛУГИ —  Разработка  AI- решений/агентов/ботов  для  автоматизации  бизнеса  —  Аудит  процессов  и  разработка  дорожной  карты  внедрения  AI  —  Консалтинг  по  разработке  и  внедрению  AI  —  Обучение  разработке  решений  на  базе  генеративного  ИИ  \nКЛЮЧЕВЫЕ\n \nКОМПЕТЕНЦИИ\n \nИ\n \nОПЫТ\n \nИИ-ассистенты  Разработка  AI- ассистентов,  повышающих  эффективность  выполнения  рабочих  задач  сотрудников\n\nИИ-МЕНЕДЖЕР\n \nПО\n \nПРОДАЖАМ\n \nИНТЕРНЕТ-МАГАЗИНА\n \nЗАПЧАСТЕЙ\n \nБЫТОВОЙ\n \nТЕХНИКИ\n  Заказчик :  Zip-KRD  —  интернет-магазин  по  продаже  запчастей  и  аксессуаров  для  бытовой  техники   Проблематика,  задача:  -  Большой  поток  клиентов  и  вопросов,  которые  менеджеры  не  успевают  \nотрабатывать\n \nв\n \nрежиме\n \n24х7\n\n-  Мгновенные  ответы  на  вопросы  клиентов  24х7,  отзывчивость  системы  -  Повышение  конверсии  обращений  в  заказы,  повышение  лояльности  клиентов  -  Экономия  ФОТ  сотрудников,  менеджеры  не  тратят  время  на  простые  ди

In [22]:
rag_conversation_chain = (
    RunnablePassthrough.assign(
        context=get_last_message_for_retriever_input | retriever | format_chunks
    )
    | conversational_answering_prompt
    | llm
    | StrOutputParser()
)

Протестируем нашу новую диалоговую цепочку с одним сообщением

In [23]:
print(question)

Вы проводите консультации?


In [24]:
# Тестируем диалог с одним сообщением
answer = rag_conversation_chain.invoke({"messages": [
    HumanMessage(content=question)
]})
print("Результат диалога:", answer)


Результат диалога: Да, мы проводим консультации по разработке и внедрению AI‑решений.  
Мы помогаем аудитировать процессы, разрабатывать дорожную карту и консультируем по всем этапам проекта.  
Если нужна более подробная информация, просто дайте знать.


Все прекрасно. Мы получили адекватный ответ на наш вопрос

Теперь зададим уточняющий вопрос "А ещё какие?" (имея ввиду, а какие еще услуги или консультации предоставляются?)

In [25]:
question2 = "А ещё какие?"

In [26]:
# Тестируем полный диалог с несколькими сообщениями
dialog_result = rag_conversation_chain.invoke({"messages": [
    HumanMessage(content=question), 
    AIMessage(content=answer),
    HumanMessage(content=question2), # "А ещё какие?"
]})
print("Результат диалога с несколькими сообщениями:", dialog_result)

Результат диалога с несколькими сообщениями: Мы также предлагаем услуги по интеграции и автоматизации бизнес‑процессов, настройке DevOps и MLOps, а также кросс‑платформенное тестирование и аудит данных.  
Кроме того, занимаемся обучением команды и разработкой пользовательских AI‑моделей с использованием PyTorch, LangChain и Haystack.  
Если нужно уточнить детали, дайте знать.


Видим проблему! Наша цепочка не смогла найти ответ на вопрос.
И действительно сам вопрос "А какие еще?" не несет в себе смысла без понимания истории диалога

### Query transformation

Мы понимаем, что чат-боты взаимодействуют с пользователями в режиме беседы и поэтому должны справляться с уточняющими вопросами.

Давайте посмотрим на чанки, которые мы получили при ответе на вопрос `А ещё что?`:

In [82]:
result_chunks = retriever.invoke(question2)

print(result_chunks[0].page_content)

AI:  Python,  PyTorch,  Haystack,  LangChain,  LangGraph  и  др.  Data:  MongoDB,  PostgreSQL,  Dremio,  Kafka  и  др.  DevOps: Docker,  Ansible,  Kubernetes,  Airﬂow  и  др.  Cloud: Yandex  Cloud,  Terraform  и  др.  
ПРОЦЕССЫ/ПРАКТИКИ
 
  Agile,  XP,  Microservices,  CI/CD,   IaC,  GitOps,  DocOps,  MLOps  
НАГРАДЫ
 
  ИИ-хакатоны  “Цифровой  прорыв”:  -  Международные  2024,  2023  -  Региональные  2024,  2023  ПРОФ- IT. Инновация  2023,2021  Премия  Рунета  2019  
СЕРТИФИКАТЫ


Видим, что чанк не содержит информации о других услугах. Значит мы были правы в своем предположении о том, что retriever не нашел нужной информации на запрос "А еще какие?"


Чтобы решить эту проблему, мы можем создать для retriever правильный поисковый запрос на основе всей истории переписки.

Применим технику Query Transformation


In [83]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

retrieval_query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Transform last user message to a search query in Russian language according to the whole conversation history above to further retrieve the information relevant to the conversation. Try to thorougly analyze all message to generate the most relevant query. The longer result better than short. Let it be better more abstract than specific. Only respond with the query, nothing else.",
        ),
    ]
)

from langchain_openai import ChatOpenAI

llm_query_transform = ChatOpenAI(model="gpt-4o", temperature=0.4)

retrieval_query_transformation_chain = retrieval_query_transform_prompt | llm_query_transform | StrOutputParser()

Мы создали цепочку для переписывания (трансформции) пользовательского сообщения в поисковый запрос для retriever. Проверим ее:

In [84]:
retrieval_query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question), #Какие услуги предоставляются?
            AIMessage(
                content=answer #Да, мы проводим консалтинг по разработке и внедрению AI.
            ),
            HumanMessage(content=question2), #А ещё какие?
        ],
    }
)

'дополнительные виды консультаций и услуг, предоставляемых в области искусственного интеллекта и технологий'

Супер! Мы видим осмысленный запрос вместо абстрактного "А какие еще?"

Давайте теперь создадим цепочку, которая будет использоваться для ответа на вопросы, учитывая историю переписки. Эта цепочка должна уметь отвечать на уточняющие вопросы:

In [85]:
rag_query_transform_chain = (
    RunnablePassthrough.assign(
        context= retrieval_query_transformation_chain | retriever | format_chunks
    )
    | conversational_answering_prompt
    | llm
    | StrOutputParser()
)

Цепочка создана. Теперь проверим ее на наших нескольких сообщениях:

In [86]:
rag_query_transform_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question), #Какие услуги предоставляются?
            AIMessage(
                content=answer #Да, мы проводим консалтинг по разработке и внедрению AI.
            ),
            HumanMessage(content=question2), #А ещё какие?
        ]
    }
)

'Кроме консалтинга, мы предлагаем разработку AI-решений, аудит процессов и обучение разработке решений на базе генеративного ИИ. Также можем предоставить услуги по внедрению ИИ в существующие системы.'

Все! Теперь у нас есть цепочка готовая для внедрения в наш бот!
Ура :)

# Advanced RAG

Практика трансформации запроса, рассмотренная нами, - это всего лишь одна из множества практик и подходов построения RAG-системы в реальной жизни для работы с реальными документами и базами знаний:

# ![RAG](slides/image_advanced.png)